# MCCE-generated water distribution in Gramicidin channel
In the following, we take a look at data generated from MCCE.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
fontsize=10
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : fontsize}

plt.rc('font', **font)
import sys
import struct
import numpy as np
import mdtraj as md
import parmed as pm
from collections import OrderedDict
from __future__ import division
from sstmap.grid_water_analysis import *
from sstmap.utils import write_watpdb_from_coords


/Users/kamranhaider/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Put together a microstate analyzer class.

In [5]:
class MicrostateAnalysis(object):
    """A class representing microstate data and various analysis methods"""
    def __init__(self, ms_data_file, head3lst_file):
        self.ms_data_file = ms_data_file
        self.byte_indices = None
        self.total_microstates = 0
        self.total_records = 0
        res_list = []
        
        with open(self.ms_data_file, "rb") as md:
            bytes_n_res = md.read(4)
            n_res = struct.unpack('i', bytes_n_res)[0]
            for i in range(n_res):
                resname = str(md.read(8))
                res_list.append(resname)
            self.n_res = n_res
        self.residue_list = res_list
        self.residue_hb_matrix = np.zeros((n_res, n_res), dtype=float)

        #with open(ms_gold_file, "r") as ms_gold:
        #    self.n_res = len([res.strip() for res in ms_gold.readlines() if len(res) != 0])
        conf_data = {}
        with open(head3lst_file, "r") as h3:
            for line in h3.readlines()[1:]:
                data = line.split()
                conf_id = int(data[0])
                conf_data[conf_id] = [data[1], data[3]]
        self.conformer_data = conf_data

    def generate_byte_indices(self, sample_frequency=100, filename=None):
        """
        Generate byte indices
        
        Parameters
        ----------
        n_res : TYPE
            Description
        sample_frequency : int, optional
            Description
        filename : None, optional
            Description
        
        Returns
        -------
        rec_indices : list
            A list of the starting bytes for each record in the microstate data file
        """
        if filename is None:
            filename = self.ms_data_file

        start_byte = 4 + (8 * self.n_res)
        bytes_per_record = (self.n_res * 2) + 20
        file_size = os.path.getsize(filename)
        # n_records = (file_size - start_byte) / bytes_per_record
        rec_indices = list(range(start_byte, file_size, sample_frequency * bytes_per_record))
        self.total_records = len(rec_indices)
        self.byte_indices = rec_indices
        
    def parse_records(self):
        """
        Parse ms.dat
        """
        trajectory = np.zeros([self.total_records, self.n_res], dtype=int)
        state_counts = np.zeros([self.total_records], dtype=int)
        energies = np.zeros([self.total_records], dtype=float)
        #print trajectory
        progress_counter = 0
        print_progress_bar(progress_counter, self.total_records)
        with open(self.ms_data_file, "rb") as ms:
            for index, record in enumerate(self.byte_indices):
                ms.seek(record)
                bytes_conf_ids = ms.read(2 * self.n_res)
                bytes_energies_1 = ms.read(8)
                ms.seek(ms.tell() + 8)
                energy = struct.unpack("d", bytes_energies_1)[0]
                bytes_state_count = ms.read(4)
                trajectory[index, :] = np.asarray(struct.unpack(str(self.n_res) + "H", bytes_conf_ids))
                #print(struct.unpack(str(self.n_res) + "H", bytes_conf_ids)[-2:])
                state_count = struct.unpack("i", bytes_state_count)[0]
                self.total_microstates += state_count
                state_counts[index] += state_count
                energies[index] += energy
                progress_counter += 1
                print_progress_bar(progress_counter, self.total_records)
        self.trajectory = trajectory
        self.state_counts = state_counts
        self.energies = energies

In [8]:
msdat = "../data/mcce_water_profiling/1_conf_per_water/ms.dat"
head3lst = "../data/mcce_water_profiling/10_conf_per_water/head3.lst"
msa = MicrostateAnalysis(msdat, head3lst)
msa.generate_byte_indices(sample_frequency=100)
msa.parse_records()
    

Progress |====================| 100.0% Done.


In [10]:
print(msa.trajectory.shape)

(870, 556)
